In [1]:
from io import StringIO
import requests
import pandas as pd
from bs4 import BeautifulSoup
from  datetime import datetime , timedelta
import bs4
import numpy as np
import time
import copy
import pandas as pd
import re

In [3]:
tick = "HDFCBANK"
nse_tick = "DMART"
amount = 100000
investment = 0
portfolio = {}
history = {}
transaction_cost = 0.0075

In [2]:
session = requests.session()
headers = {
    "user-agent": "Chrome/87.0.4280.88"
}
head = {
    'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/87.0.4280.88 Safari/537.36 "
}

def getHistoryData(company, from_date=(datetime.today().strftime("%d-%m-%Y")), to_date=(datetime(datetime.today().year - 1, datetime.today().month,datetime.today().day).strftime("%d-%m-%Y"))):
    session.get("https://www.nseindia.com", headers=head)
    session.get("https://www.nseindia.com/get-quotes/equity?symbol=" + company, headers=head) 
    session.get("https://www.nseindia.com/api/historical/cm/equity?symbol="+company, headers=head)
    url = "https://www.nseindia.com/api/historical/cm/equity?symbol=" + company + "&series=[%22EQ%22]&from=" + from_date + "&to=" + to_date + "&csv=true"
    webdata = session.get(url=url, headers=head)
    df = pd.read_csv(StringIO(webdata.text[3:]))
    return df


def get_intrinsic_value(tick):
    url = "https://www.screener.in/company/"+tick+"/consolidated/"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.content, 'html.parser')
    section = soup.find('section', id='profit-loss')
    table = section.find('table', class_='data-table responsive-text-nowrap')
    thead = table.find('thead')
    tbody = table.find('tbody')

    headers = [header.get_text(strip=True) for header in thead.find_all('th')]

    data_rows = []
    for row in tbody.find_all('tr'):
        row_data = [cell.get_text(strip=True) for cell in row.find_all(['td', 'th'])]
        data_rows.append(row_data)

    df = pd.DataFrame(data_rows, columns=headers)

    dates = list(df.columns)[1:len(df.columns)-1]
    dates = [['1-03-' +date.split(' ')[1], '10-03-' +date.split(' ')[1] ] for date in dates]

    new_dates = []
    price = []
    for i in dates:
        try:
            x = getHistoryData(tick,from_date=i[0],to_date=i[1])['OPEN '].iloc[0]
            new_dates.append(i)
            x = str(x)
            x = x.replace(",", "")
            x = x.split('.')[0]
            price.append(int(x))
        except:
            continue


    temp = list(df.loc[df[''] == 'EPS in Rs'].iloc[0])[1:]
        
    eps = []
    for i in temp:
        eps.append(int(i.split(".")[0]) + (int(i.split(".")[1])/(10**len(i.split(".")[1]))))

    ttm = eps[-1]
    eps = eps[:-1]

    head = df.columns[1:-1]
    temp = []
    for i in head:
        temp.append(i[-4:])
    
    rev_eps = []
    for i in range(len(temp)):
        for j in new_dates:
            if temp[i] in j[0]:
                rev_eps.append(eps[i])
    
    eps = rev_eps
    pe = np.array(price)/np.array(eps)

    stable = True
    slope = np.cov(np.arange(len(eps)), eps)[0, 1] / np.var(np.arange(len(eps)))

    if slope>0.75:
        stable = True
    else :
        stable = False

    if stable == True:
        eps_gr = (eps[-1]/eps[0])**(1/len(eps))*100
        pe_avg = np.average(pe)
        pe_gr = (pe[-1]/pe[0])**(1/len(pe))*100
        intrinsic_value = pe_avg * ttm
        return intrinsic_value
    
    else:
        return "Unstable"


In [4]:
nse = [["DMART", "AVEU"], ["ABB", "ABB"], ["HDFCBANK", "HDBK"], ["BHEL", "BHEL"]]

In [3]:
def get_price(ticker):
    url = f'https://www.tickertape.in/stocks/{ticker}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    class1 = "jsx-3168773259 current-price typography-h1 text-primary"
    price = soup.find(class_ = class1).text
    price = price.replace(",", "")
    price = price.split('.')
    ret = int(price[0]) + int(price[1])/(10**len(price[1]))
    return ret

def buy(ticker, quantity):
    price = get_price(ticker)
    global amount, investment, history
    if amount > price*quantity*(1+transaction_cost):
        amount = amount - price*quantity - transaction_cost*price*quantity
        portfolio[ticker] = portfolio.get(ticker, 0) + quantity
        history[ticker+"+"] = price*quantity
        investment = price*quantity + investment

def sell(ticker, quantity):
    global amount, investment
    price = get_price(ticker)
    amount = amount + price*quantity*(1-transaction_cost)
    portfolio[ticker] = portfolio[ticker] - quantity
    history[ticker+"-"] = price*quantity
    investment = investment - price*quantity

def refresh():
    global investment, portfolio
    x = 0
    for i in portfolio:
        x = x + get_price(i)*portfolio[i]
    investment = x

def get_news(tick):
    url = "https://www.tickertape.in/stocks/" + tick + "#news"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        news_table = soup.find(class_ = 'latest-news-holder')    
        news_div = soup.find('div', {'id': 'news'})
        older_news_div = news_div.find('div', {'data-section-tag': 'olderNews'})
        news_divs = older_news_div.select('div.jsx-1359472481 div.jsx-3953764037')
        ret1 = []
        ret2 = []
        no = []
        for name_box in news_table.find_all('p', class_='shave-root'):
            ret1.append(name_box.text.strip())
        for index, news_div in enumerate(news_divs):
            title_element = news_div.select_one('h5.news-title')
            news_title = ret2.append(title_element.text.strip()) if title_element else no.append(1)
        
        x = list(set(ret1)) + list(set(ret2))
        news = ""
        for i in x:
            news = news+i+" "
        return news

In [7]:
stocks = pd.read_csv("/Users/satyam/Desktop/Projects/ACM/GPT-Fund/tick_final.csv")
ticklist = list(stocks.tt[:])
nselist = list(stocks.nse[:])

In [18]:
# results = {}

In [8]:
# from tqdm import tqdm
# for i in tqdm(range(len(ticklist))):
#     try:
#         results[ticklist[i]] = [get_intrinsic_value(nselist[i]), get_price(ticklist[i])]

#     except:
#         print("Error for Stock: ", nselist[i], " at pos ", i)

In [31]:
# iv = []
# price = []
# for i in results:
#     iv.append(results[i][0])
#     price.append(results[i][1])
# df = pd.DataFrame({'Stock': results.keys(), 'Intrinsic_Value': iv, 'Price': price})


In [34]:
stocks

,Unnamed: 0,Stock,Intrinsic_Value,Price
0,0,ABB,Unstable,5841.35
1,1,ADAI,601.8401498820529,1063.75
2,2,ADEL,Unstable,3207.35
3,3,ADNA,4561.2249818719265,1930.30
4,4,APSE,768.9661002074058,1326.95
...,...,...,...,...
83,83,UNBK,Unstable,153.75
84,84,VARB,3344.00146566806,1414.25
85,85,VDAN,68.52418418542165,275.35
86,86,WIPR,687.370950841566,514.55


In [10]:
stocks = pd.read_csv("stocks_saved.csv")

In [46]:
from openai import OpenAI

openai = OpenAI(
    api_key="bPdeUDQV6WUDGE6ifWQHult7iii7OWFj",
    base_url="https://api.deepinfra.com/v1/openai",
)

In [50]:
from tqdm import tqdm

news = []
for i in tqdm(range(len(stocks.Stock))):
    name = list(stocks.Stock)[i]
    a = get_news(name)
    prompt = f"Model Question: For the given stock, I need you to take the role of a financial analyst. You will be given a set of news from a certain stock. I need you to give me a summary of the News that you are fed. Second, based on the news, give me a risk measure for the stock from either of - Low, Moderate and High. Thirdly, recommend me whether to Buy, Sell or Hold the stock. Lastly, give the stock a rating out of 10. Example News: NCLAT Dismisses BHEL’s Pleas Against REC Over Insolvency Of Hiranmaye Energy Coal Gasification Scheme: Govt Seeks Feedback By March 20, Check Details - News18 Stocks to Watch: Adani Green, ZEE, Eicher Motors, Jana SFB, BHEL, Oil India, and Others - News18 BHEL Shares Rally 14% to Hit 52-Week High; Know Why It Is Rising Today? - News18 MSCI rejig tomorrow: 5 PSUs likely to be added to standard index, India could see $1 billion inflow Stocks to Watch: HCL Tech, Wipro, Jio Fin, LIC, DMart, LIC, BHEL, Indigo, and Others - News18 BHEL Shares Jump 14% To 12-Year High After Three Large Trades BHEL reports ₹149 crore net loss in Q3 Stocks To Watch: LIC, RVNL, Biocon, Tata Power, Hero MotoCorp, IRFC, BHEL, Telecom Companies. Answer: Summary: BHEL shares have experienced significant fluctuations, rallying to a 52-week high following multiple positive developments including a jump in share price, potential inclusion in the MSCI index, and large trades. However, the company reported a net loss in Q3 and faces legal challenges related to insolvency proceedings against Hiranmaye Energy. Risk Measure: Moderate. The risk level is moderate due to the mixed nature of news surrounding BHEL. Positive factors such as the rally in share price and potential index inclusion are countered by the company's reported net loss in Q3 and legal disputes. Recommendation: Hold. Given the mixed signals in the news, it's advisable for investors to hold their positions on BHEL until there is more clarity regarding its financial performance and legal challenges. ⁠Rating: 6/10. The stock shows promise with recent rallies and potential index inclusion, but uncertainties regarding financial losses and legal issues warrant a cautious approach. Perform a similar analysis for the stock {name}, the news of which you are provided"

    chat_completion = openai.chat.completions.create(
        model="mistralai/Mistral-7B-Instruct-v0.1",
        messages=[
            {"role": "system", "content": a},
            {"role": "user", "content": prompt},
        ],
        temperature=0.5
    )

    news.append(chat_completion.choices[0].message.content)

100%|██████████| 88/88 [25:08<00:00, 17.15s/it]


In [53]:
import pickle

with open('news_.pkl', 'wb') as f:
    pickle.dump(news, f)

In [60]:
with open('news_.pkl', 'rb') as f:
    loaded_list = pickle.load(f)



In [112]:
import re

def extract_information(summary):
    risk_measure_pattern = r"Risk Measure: (\w+)\."
    recommendation_pattern = r"Recommendation: (\w+)\."
    rating_pattern = r"Rating: (\d+)/\d+"

    risk_measure_match = re.search(risk_measure_pattern, summary)
    recommendation_match = re.search(recommendation_pattern, summary)
    rating_match = re.search(rating_pattern, summary)

    risk_measure = risk_measure_match.group(1) if risk_measure_match else None
    recommendation = recommendation_match.group(1) if recommendation_match else None
    rating = rating_match.group(1) if rating_match else None

    return risk_measure, recommendation, int(rating)

In [118]:
risk = []
score = []
recomm = []

In [178]:
lu1 = {"Low": 0, "Moderate": 1, "High": 2}
lu2 = {"Sell": 0, "Hold": 1, "Buy": 2}
# Risk 0=Low, 1=Moderate, 2=High
# Recom 0=Sell, 1=Hold, 2=Buy

for i in range(88,len(loaded_list)):
    r,re,s = extract_information(loaded_list[i])
    risk.append(lu1[r])
    score.append(s)
    recomm.append(lu2[re])
    

In [180]:
risk2 = []
recomm2 = []
for i in risk:
    if i in lu1.keys():
        risk2.append(lu1[i])
    else:
        risk2.append(i)

for i in recomm:
    if i in lu2.keys():
        recomm2.append(lu2[i])
    else:
        recomm2.append(i)

In [181]:
stocks

,Unnamed: 0,Stock,Intrinsic_Value,Price
0,0,ABB,Unstable,5841.35
1,1,ADAI,601.8401498820529,1063.75
2,2,ADEL,Unstable,3207.35
3,3,ADNA,4561.2249818719265,1930.30
4,4,APSE,768.9661002074058,1326.95
...,...,...,...,...
83,83,UNBK,Unstable,153.75
84,84,VARB,3344.00146566806,1414.25
85,85,VDAN,68.52418418542165,275.35
86,86,WIPR,687.370950841566,514.55


In [182]:
stocks["Risk"] = risk2
stocks["Recomm"] = recomm2
stocks["Score"] = score

In [261]:
stocks.to_csv("stocks_saved.csv")

In [191]:
iv = list(stocks.Intrinsic_Value)
price = list(stocks.Price)

uv = []
for i in range(len(iv)):
    if iv[i] == "Unstable":
        uv.append(-1)
    else:
        uv.append(float(iv[i])/float(price[i]))

stocks['Value'] = uv

In [6]:
def number_of_stocks(amt, price):
    buy = [0 for j in range(len(price))]
    i = 0
    while amt>price[0]:
        if amt > price[i]:
            buy[i] += 1
            amt = amt - price[i]
        i = (i+1)%len(price)
    return buy

In [4]:
stocks = pd.read_csv("stocks_saved.csv")

In [253]:
# Risk = High - 2 Stock options 
amt = 1111
high = stocks.loc[(stocks['Risk'] == 2) | (stocks['Value'] > 7)]
high_sorted = high.sort_values(by='Value',  ascending=False)

price_list = []
num = 0
while num < 2:
    if high_sorted.iloc[num]['Price'] < amt:
        price_list.append(high_sorted.iloc[num]['Price'])
        num = num + 1

In [13]:
# Risk = Moderate - 1 High, 1 Low, 2 Moderate
amt = 11111
high = stocks.loc[(stocks['Risk'] == 2) | (stocks['Value'] > 7)]
high_sorted = high.sort_values(by='Value',  ascending=False)
mod = stocks.loc[(stocks['Risk'] == 1) & (stocks['Value'] < 7)]
mod_sorted = mod.sort_values(by='Value',  ascending=False)
low = stocks.loc[(stocks['Risk'] == 0) & (stocks['Value'] < 7)]
low_sorted = low.sort_values(by='Value',  ascending=False)

price_list = []
num = 0
while num < 2:
    if mod_sorted.iloc[num]['Price'] < amt:
        price_list.append(mod_sorted.iloc[num]['Price'])
        print(mod_sorted.iloc[num]['Stock'])
        num = num + 1

num = 0
while num < 1:
    if high_sorted.iloc[num]['Price'] < amt:
        price_list.append(high_sorted.iloc[num]['Price'])
        print(high_sorted.iloc[num]['Stock'])
        num = num + 1

num = 0
while num < 1:
    if low_sorted.iloc[num]['Price'] < amt:
        price_list.append(low_sorted.iloc[num]['Price'])
        print(low_sorted.iloc[num]['Stock'])
        num = num + 1

number_of_stocks(amt, price_list)

HDBK
VARB
IOC
BJFS


[3, 2, 2, 2]

In [17]:
get_price('BJFS')

1598.8

In [260]:
# Risk = Low - 3 Lows
amt = 11111

low = stocks.loc[(stocks['Risk'] == 0)]
low_sorted = low.sort_values(by='Value',  ascending=False)

price_list = []
num = 0
while num < 3:
    if low_sorted.iloc[num]['Price'] < amt:
        price_list.append(low_sorted.iloc[num]['Price'])
        num = num + 1

number_of_stocks(amt, price_list)

[7, 3, 2]

In [263]:
stocks = pd.read_csv("/Users/satyam/Desktop/Projects/ACM/GPT-Fund/stocks_saved.csv")
names = pd.read_csv("/Users/satyam/Desktop/Projects/ACM/GPT-Fund/tick_final.csv")


In [264]:
names.loc[names['tt'] == "ABB", 'stock'].values[0]

'ABB India Ltd'

In [265]:
low = stocks.loc[(stocks['Risk'] == 0)]
low_sorted = low.sort_values(by='Value',  ascending=False)
low_sorted

,Unnamed: 0,Stock,Intrinsic_Value,Price,Risk,Recomm,Score,Value
45,45,IOC,1807.2348912339285,174.25,0,2,8,10.371506
70,70,SBI,7789.069303116493,773.70,0,1,8,10.067299
30,30,EICH,29011.21640952016,3783.90,0,2,8,7.667015
19,19,BPCL,4643.898911695731,626.40,0,2,8,7.413632
14,14,BJFS,11137.74073717562,1598.80,0,2,9,6.966313
31,31,GAIL,600.090664238577,183.25,0,1,8,3.274710
47,47,INFY,3818.1831616505615,1600.60,0,2,8,2.385470
23,23,CHLA,2496.5275527379886,1095.45,0,2,8,2.278997
42,42,ICBK,2203.063656389878,1078.70,0,2,8,2.042332
53,53,KTKM,3392.136921178156,1729.65,0,2,8,1.961170


In [7]:
get_price("IOC")

174.25

In [2]:
import pandas as pd
mf = pd.read_csv("/Users/satyam/Desktop/Projects/ACM/GPT-Fund/mf_saved.csv")

In [3]:
low = mf.loc[(mf['beta'] < 0.9)]
mod = mf.loc[(mf['beta'] > 0.9) & (mf['beta'] < 1)]
high = mf.loc[(mf['beta'] > 1)]

In [4]:
low

,Unnamed: 0,std,beta,sharpe,treynor,alpha,cagr2,cagr5,mf
4,4,13.43,0.89,0.21,0.03,-6.82,9.27,15.20,Uti Flexi Cap Fund Direct Plan
5,5,13.43,0.89,0.21,0.03,-6.82,8.41,14.37,Uti Flexi Cap Fund Regular Plan
18,18,14.83,0.83,1.38,0.25,6.53,36.27,27.67,Bank Of India Manufacturing Infrastructure Fun...
19,19,14.83,0.83,1.38,0.25,6.53,34.52,26.03,Bank Of India Manufacturing Infrastructure Fun...
20,20,15.08,0.64,1.47,0.35,3.51,38.88,27.25,Invesco India Infrastructure Fund
21,21,15.08,0.64,1.47,0.35,3.51,40.90,29.03,Invesco India Infrastructure Fund Direct Plan
31,31,1.93,0.38,-0.57,-0.03,-0.81,7.50,8.83,Dsp Government Securities Fund Direct Plan
32,32,1.93,0.38,-0.57,-0.03,-0.81,6.91,8.24,Dsp Government Securities Fund Regular Plan


In [24]:
def rank_up(df):
    df_copy = df.copy()
    df_copy["CAGR2 Rank"] = df_copy["cagr2"].rank(ascending=True)
    df_copy["CAGR5 Rank"] = df_copy["cagr5"].rank(ascending=True)
    df_copy["sharpe_rank"] = df_copy["sharpe"].rank(ascending=True)
    df_copy["alpha_rank"] = df_copy["alpha"].rank(ascending=True)
    df_copy["treynor_rank"] = df_copy["treynor"].rank(ascending=True)

    df_copy["Weighted Score"] = (
        0.35 * df_copy["CAGR2 Rank"]
        + 0.25 * df_copy["CAGR5 Rank"]
        + 0.20 * df_copy["sharpe_rank"]
        + 0.10 * df_copy["alpha_rank"]
        + 0.10 * df_copy["treynor_rank"]
    )
    df_copy["Rank"] = df_copy["Weighted Score"].rank(ascending=False)
    return df_copy[["mf", "Weighted Score", "Rank"]].sort_values(by="Rank")


In [21]:
df_copy = low.copy()

df_copy["CAGR5 Rank"] = df_copy["cagr5"].rank(ascending=True)


In [27]:
rank_up(high)

,mf,Weighted Score,Rank
22,Canara Robeco Equity Hybrid Fund Direct Plan G...,6.30,1.0
23,Canara Robeco Equity Hybrid Fund Regular Plan ...,5.70,2.0
29,Mirae Asset Short Term Fund Direct Plan,4.30,3.0
28,Edelweiss Banking And Psu Debt Fund Regular Plan,3.95,4.0
33,Bank Of India Ultra Short Duration Fund Direct...,2.85,5.0
30,Mirae Asset Short Term Fund Regular Plan,2.65,6.0
34,Bank Of India Ultra Short Duration Fund Regula...,2.25,7.0


In [23]:
df_copy

,Unnamed: 0,std,beta,sharpe,treynor,alpha,cagr2,cagr5,mf,CAGR5 Rank
4,4,13.43,0.89,0.21,0.03,-6.82,9.27,15.20,Uti Flexi Cap Fund Direct Plan,4.0
5,5,13.43,0.89,0.21,0.03,-6.82,8.41,14.37,Uti Flexi Cap Fund Regular Plan,3.0
18,18,14.83,0.83,1.38,0.25,6.53,36.27,27.67,Bank Of India Manufacturing Infrastructure Fun...,7.0
19,19,14.83,0.83,1.38,0.25,6.53,34.52,26.03,Bank Of India Manufacturing Infrastructure Fun...,5.0
20,20,15.08,0.64,1.47,0.35,3.51,38.88,27.25,Invesco India Infrastructure Fund,6.0
21,21,15.08,0.64,1.47,0.35,3.51,40.90,29.03,Invesco India Infrastructure Fund Direct Plan,8.0
31,31,1.93,0.38,-0.57,-0.03,-0.81,7.50,8.83,Dsp Government Securities Fund Direct Plan,2.0
32,32,1.93,0.38,-0.57,-0.03,-0.81,6.91,8.24,Dsp Government Securities Fund Regular Plan,1.0
